In [143]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [213]:
url  = "https://sale.alibaba.com/p/rank/list.html?spm=a2700.product_home_newuser.scenario_overview.topRanking&wx_navbar_transparent=true"

In [145]:
# Récuperer la page source 

page_source = requests.get(url)

In [146]:
page_source.text

'<!doctype html>\n<html dir="ltr">\n <head>\n  <script> window.__BB_time1 = Date.now(); </script>\n  <meta charset="utf-8">\n  <meta name="data-spm" content="a27aq">\n  <meta name="viewport" content="width=device-width,user-scalable=no,initial-scale=1,maximum-scale=1,minimum-scale=1,viewport-fit=cover">\n  <title>Top ranking</title>\n  <meta name="keywords" content="">\n  <meta name="description" content="">\n  <meta name="pid" content="3012651">\n  <meta name="serverTime" content="1702994323708">\n  <meta name="sys" content="ncms">\n  <meta name="tplid" content="3012651">\n  <meta name="env" content="production">\n  <meta name="referrer" content="no-referrer-when-downgrade">\n  <link rel="dns-prefetch" href="//s.alicdn.com">\n  <link rel="dns-prefetch" href="//insights.alibaba.com">\n  <link rel="shortcut icon" href="https://s.alicdn.com/@icon/favicon-v1.ico" type="image/x-icon">\n  <meta property="og:image" content="https://s.alicdn.com/@img/tfs/TB1v02kzoT1gK0jSZFrXXcNCXXa-200-200.pn

In [147]:
# Faire le parsing pour permettre la structuration du document HTML facilitant ainsi l'indexation, la recherche et la manipulation des informations

soup = BeautifulSoup(page_source.text,"html.parser")

In [148]:
soup

<!DOCTYPE html>

<html dir="ltr">
<head>
<script> window.__BB_time1 = Date.now(); </script>
<meta charset="utf-8"/>
<meta content="a27aq" name="data-spm"/>
<meta content="width=device-width,user-scalable=no,initial-scale=1,maximum-scale=1,minimum-scale=1,viewport-fit=cover" name="viewport"/>
<title>Top ranking</title>
<meta content="" name="keywords"/>
<meta content="" name="description"/>
<meta content="3012651" name="pid"/>
<meta content="1702994323708" name="serverTime"/>
<meta content="ncms" name="sys"/>
<meta content="3012651" name="tplid"/>
<meta content="production" name="env"/>
<meta content="no-referrer-when-downgrade" name="referrer"/>
<link href="//s.alicdn.com" rel="dns-prefetch"/>
<link href="//insights.alibaba.com" rel="dns-prefetch"/>
<link href="https://s.alicdn.com/@icon/favicon-v1.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="https://s.alicdn.com/@img/tfs/TB1v02kzoT1gK0jSZFrXXcNCXXa-200-200.png" property="og:image"/>
<meta content="website" property="o

In [191]:
# Rechercher tous les éléments se trouvant dans la classe spécifiée
elements = soup.find_all("div", class_ = "hugo4-pc-grid hugo4-pc-grid-3 hugo4-pc-grid-list test-class")

In [291]:
def get_data(url):
    """Cette fonction prend en argument une url et 
    renvoie un dataFrame contenant le nom,le nombre de vues 
    ou de commandes ces 30 derniers jours 
      et le prix du produit""""
    
    page_source = requests.get(url)
    soup = BeautifulSoup(page_source.text,"html.parser")
    my_data = {}
    elements = soup.find_all("div", class_ = "hugo4-pc-grid hugo4-pc-grid-3 hugo4-pc-grid-list test-class")
    for elt in elements:
        for i in range(len(elt.find_all("div", class_="hugo4-pc-grid-item"))):
            name = elt.find_all("div", class_="hugo4-pc-grid-item")[i].find_all("div",class_="hugo3-util-ellipsis title hugo3-fw-bold hugo3-fz-large line-2")[0].text.strip()
            order = elt.find_all("div", class_="hugo4-pc-grid-item")[i].find_all("span",class_="customize-text")
            price=  elt.find_all("div", class_="hugo4-pc-grid-item")[i].find_all("span", class_="price-text pc")
            my_data[f"#{i}"]= {"name":name,"order":[ordt.text for ordt in order],"price":[prix.text for prix in price]}
    order = [v["order"][0] if len(v["order"]) else "-" for k,v in my_data.items()]
    price = [max(v["price"][0].split("-")) for k,v in my_data.items()]
    #price = [elt for prix in price for elt in prix]
    dict_data = {"name":[v["name"] for k,v in my_data.items()],"nborder_or_nbviews":order,"price":price}
    df = pd.DataFrame(dict_data)  
    
    return df

In [292]:
get_data(url)

,name,nborder_or_nbviews,price
0,Quartz Watches With Chronograph,-,$0.50
1,Pumps For Women,1K+,$7.50
2,Plate,1K+,$6.88
3,Men's Stand Jackets,5M+,$9.10
4,Sport Mechanical Watches,1.6M+,$38.40


In [286]:
data.items()

<generator object DataFrame.items at 0x7fd066fc6e30>

In [229]:
dict_data = {"name":[v["name"] for k,v in data.items()],"order":[v["order"] for k,v in data.items()],"price":[v["price"] for k,v in data.items()]}

In [252]:
dict_data["order"][1][0]

'1K+ '

In [231]:
pd.DataFrame(dict_data)

,name,order,price
0,Quartz Watches With Chronograph,[],"[$0.50 - $7.90, $10.00 - $11.80, $10.00 - $11.80]"
1,Pumps For Women,[1K+ ],"[$7.50 - $8.80, $6.69 - $7.59, $30.00 - $68.00]"
2,Plate,[1K+ ],"[$6.88 - $16.68, $9.00 - $15.67, $0.28 - $0.40]"
3,Men's Stand Jackets,[5M+ ],"[$9.10 - $18.20, $4.96 - $5.36, $8.10 - $16.20]"
4,Sport Mechanical Watches,[1.6M+ ],"[$38.40 - $43.20, $79.00 - $84.00, $26.00 - $2..."


In [273]:
my_data['#1']["price"][0].split("-")

['$7.50 ', ' $8.80']